In [1]:
from surprise import Dataset, SVD, SVDpp, NMF
from surprise.model_selection import cross_validate, GridSearchCV


In [2]:
# Завантажимо датасет, створимо модель та виконаємо кросвалідацію з розбивкою
# датасету на 5 фрагментів. Для цього візьмемо приклад з документації:


# Load the movielens-100k dataset (download it if needed),
data = Dataset.load_builtin("ml-100k")

# We'll use the famous SVD algorithm.
algo = SVD()

# Run 5-fold cross-validation and print results
cross_validate(algo, data, measures=["RMSE", "MAE"], cv=5, verbose=True, n_jobs=-1)

# Наче все працює. Модель створена, отримані результати. Оцінка результатів проводиться по метрикам RMSE та MAE

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9321  0.9396  0.9357  0.9339  0.9399  0.9363  0.0031  
MAE (testset)     0.7363  0.7407  0.7365  0.7359  0.7408  0.7380  0.0022  
Fit time          1.00    1.19    1.11    1.05    1.01    1.07    0.07    
Test time         0.30    0.23    0.24    0.19    0.16    0.22    0.05    


{'test_rmse': array([0.93214979, 0.93957071, 0.93574921, 0.93394027, 0.93994682]),
 'test_mae': array([0.73625688, 0.74069726, 0.73649147, 0.73585409, 0.74076342]),
 'fit_time': (0.9961390495300293,
  1.19187593460083,
  1.1109328269958496,
  1.051797866821289,
  1.013418197631836),
 'test_time': (0.3006935119628906,
  0.2315220832824707,
  0.23623418807983398,
  0.1925945281982422,
  0.16035246849060059)}

In [3]:
# Для вивчення впливу параметрів на роботу алгоритму скористаємось GridSearchCV.
# Дослідимо вплив наступних параметрів: n_factors, n_epochs


param_grid = {"n_factors": range(10, 101, 10),
              "n_epochs": range(20, 51, 10)
}

gs = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], cv=5, n_jobs=-1)

gs.fit(data)

# best RMSE score
print(gs.best_score["rmse"])

# combination of parameters that gave the best RMSE score
print(gs.best_params["rmse"])

# Бачимо, що найкращий результат отримано при кількості епох 30, кількості факторів 10.

0.9316860957811859
{'n_factors': 10, 'n_epochs': 30}


In [4]:
# По аналогії, скористаємось GridSearchCV для дослідження інших алгоритмів:
# SVD++:

param_grid = {"n_factors": range(10, 31, 5),
              "n_epochs": range(10, 31, 10)
}

gs = GridSearchCV(SVDpp, param_grid, measures=["rmse", "mae"], cv=5, n_jobs=-1)

gs.fit(data)

# best RMSE score
print(gs.best_score["rmse"])

# combination of parameters that gave the best RMSE score
print(gs.best_params["rmse"])


0.9185048232085323
{'n_factors': 10, 'n_epochs': 20}


In [5]:
# NMF:

param_grid = {"n_factors": range(5, 41, 5),
              "n_epochs": range(30, 71, 10)
}

gs = GridSearchCV(NMF, param_grid, measures=["rmse", "mae"], cv=5, n_jobs=-1)

gs.fit(data)

# best RMSE score
print(gs.best_score["rmse"])

# combination of parameters that gave the best RMSE score
print(gs.best_params["rmse"])


0.9497084560656056
{'n_factors': 40, 'n_epochs': 70}


In [6]:
# Висновок: алгоритм SVD++  при кількості епох 20, кількості факторів 10 забезпечив
# кращий результат. Алгоритми SVD та NMF забезпечили порівняний результат,
# але при формальному порівнянні по метриці RMSE видали гірший результат.